<h1> Example of TabularDataset Index Issue </h1>

Azure's TabularDataset implementation introduces an index, \_\_index\_level_0\_\_ when creating or reading parquet files that were originally written by Pandas/Python.  This occurs when an index is unnamed but has been modified at some point; if an index is named we get an extra column with the same name as the index.

When making changes to datasets, this additional field causes Azure errors if not handled.  Depending on what's been done to the index of the original dataset, you may or may not get that additional field.

This notebook demonstrates those errors, which are unexpected when performing simple operations.  This code was originally run on Python3.6 in AzureML.

<h2> Pure Pandas Behavior </h2>
This section demonstrates that native Pandas functions are unaffected by the indexing issues.  When I ran this Notebook I had Pandas version 0.25.3, Numpy 1.18.5.   

The issue seems to occur with a non-default index.  If you have a named index, Azure will make it into a column rather than an index (pandas does not have this problem).  If your index is unnamed then you get errors related to df.index.name

In [54]:
import pandas as pd
pd.__version__

'0.25.3'

In [55]:
import numpy as np
np.__version__

'1.18.5'

In [56]:
# Directory into which you want to save the parquet files
base_dir = os.getenv('HOME')

In [64]:
# Create an example dataframe - reindex it and make the index unnamed
start_df = pd.DataFrame.from_dict({'i': list(range(200, 300)),
                                   'x': list(range(100)), 
                                   'y': np.random.randint(0,100,size=100)}) \
    .set_index('i') \
    .rename_axis(index=None)
start_df.head()

,x,y
200,0,34
201,1,53
202,2,61
203,3,32
204,4,49


In [65]:
# Save the data to parquet
start_df.to_parquet(os.path.join(base_dir, 'init.parquet'))

In [60]:
# Retrieve from parquet.  Verify no chanves
start2_df = pd.read_parquet(os.path.join(base_dir, 'init.parquet'))
start2_df.dtypes

x    int64
y    int64
dtype: object

In [47]:
# Sample rows and save to parquet again
end_df_correct = start_df.sample(5)
end_df_correct.to_parquet(os.path.join(base_dir, 'fin_pd.parquet'))

In [48]:
# Read from parquet
fin2_pd = pd.read_parquet(os.path.join(base_dir, 'fin_pd.parquet'))
fin2_pd.dtypes

x    int64
y    int64
dtype: object

Everything is fine.  I am able to read from parquet, modify, and write again with no strange effects

<h2> Azure Setup </h2>
I want to show the same series of operations via the Azure TabularDataset, but first I need to do some setup for the Azure Machine Learning environment.  You will need to use your own workspace, blob storage, etc. 

In [ ]:
# Connect to workspace - use your own info here
from azureml.core.workspace import Workspace

ws = Workspace.get(name='YOUR-INFO-HERE', # Put in your own info
               subscription_id="YOUR-INFO-HERE",
               resource_group="YOUR-INFO-HERE")

In [23]:
# connect to a datastore - use your own info here
from azureml.core import Datastore
output_datastore = Datastore.get(ws, 'YOUR-INFO-HERE')

In [26]:
# Set a target path on your datastore where you want to save files
output_path = 'YOUR-INFO-HERE'

<h2> TabularDataSet Behavior </h2>
In this section I show a failure when the same set of operations is performed using the Azure TabularDataset wrapper

In [66]:
# Move the parquet file to the datastore
output_datastore.upload_files(files=[os.path.join(base_dir, 'init.parquet')],
                              target_path=output_path,
                              overwrite=True)

Uploading an estimated of 1 files
Uploading /home/azureuser/init.parquet
Uploaded /home/azureuser/init.parquet, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_dc664804225e4c4fa2c56667bca50c9d

In [67]:
# Make the parquet file into a dataset
from azureml.core import Dataset

dataset_init =  Dataset.Tabular.from_parquet_files(path=[(output_datastore,  
                                                           '/'.join([output_path, 'init.parquet']))])   

In [68]:
# Read the dataset into Pandas
init_from_ds = dataset_init.to_pandas_dataframe()

In [69]:
# Important!  Note the index column, __index_level_0__ , has been added!  
# If the index had a name that name would have been used.
init_from_ds.dtypes

x                    int64
y                    int64
__index_level_0__    int64
dtype: object

In [70]:
# Sample the dataframe, convert to parquet
fin_from_ds = init_from_ds.sample(5)
fin_from_ds.to_parquet(os.path.join(base_dir, 'fin_from_ds_pd.parquet'))

In [71]:
# Upload the final dataset to blob storage
output_datastore.upload_files(files=[os.path.join(base_dir, 'fin_from_ds_pd.parquet')],
                              target_path=output_path,
                              overwrite=True)

Uploading an estimated of 1 files
Uploading /home/azureuser/fin_from_ds_pd.parquet
Uploaded /home/azureuser/fin_from_ds_pd.parquet, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_853121932b5d484ea45ef172d31335a3

<h3> Error step - try to make a new dataset from this parquet file </h3>

In [72]:
dataset_fin_from_ds =  Dataset.Tabular.from_parquet_files(path=[(output_datastore,  
                                                           '/'.join([output_path, 'fin_from_ds_pd.parquet']))]) 

DatasetValidationError: DatasetValidationError:
	Message: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
ScriptExecutionException was caused by StreamAccessException.
  Failed to read Parquet file at: [REDACTED]
    Current parquet file is not supported.
      Field '[REDACTED]' was already present in the record.
| session_id=cd6fee42-0d07-4575-b31c-7d20df74ec0d
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot load any data from the specified path. Make sure the path is accessible and contains data.\nScriptExecutionException was caused by StreamAccessException.\n  Failed to read Parquet file at: [REDACTED]\n    Current parquet file is not supported.\n      Field '[REDACTED]' was already present in the record.\n| session_id=cd6fee42-0d07-4575-b31c-7d20df74ec0d"
    }
}

<h3> Another Error Path </h3>
It seems that if I set validate=False in the dataset creation, that step succeeds, but then I have an issue with the to_pandas_dataframe() method

In [73]:
dataset_fin_from_ds2 =  Dataset.Tabular.from_parquet_files(path=[(output_datastore,  
                                                           '/'.join([output_path, 'fin_from_ds_pd.parquet']))],
                                                          validate=False) 

In [74]:
dataset_fin_from_ds2.to_pandas_dataframe()

UserErrorException: UserErrorException:
	Message: Execution failed with error message: ScriptExecutionException was caused by StreamAccessException.
  Failed to read Parquet file at: [REDACTED]
    Current parquet file is not supported.
      Field '[REDACTED]' was already present in the record.
| session_id=cd6fee42-0d07-4575-b31c-7d20df74ec0d ErrorCode: ScriptExecution.StreamAccess.Validation.FieldConflict
	InnerException 
Error Code: ScriptExecution.StreamAccess.Validation.FieldConflict
Validation Error Code: NotSupported
Validation Target: ParquetFile
Failed Step: b9cbf103-a978-4ecb-bb54-ec6246fa6adc
Error Message: ScriptExecutionException was caused by StreamAccessException.
  Failed to read Parquet file at: /vcarey/output_data/20210121_parquet_test/fin_from_ds_pd.parquet
    Current parquet file is not supported.
      Field '__index_level_0__' was already present in the record.
| session_id=cd6fee42-0d07-4575-b31c-7d20df74ec0d
	ErrorResponse 
{
    "error": {
        "code": "Us